<a href="https://colab.research.google.com/github/BankNatchapol/Comparison-Of-Quantum-Gradient/blob/main/discrete_vs_continuous/discrete_gradient_comparison.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pennylane-sf

In [2]:
import pennylane as qml
from pennylane import numpy as np
from pennylane.optimize import GradientDescentOptimizer

import pandas as pd

import networkx as nx
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import plotly.express as px

# **Gradient methods**

In [53]:
#@title Auxilary function
import warnings
warnings.filterwarnings("ignore")

param_gates = ['RX', 'RY', 'RZ']

def padding(params):
    j = 0
    padded = []
    for i in range(len(ansatz_lcu)):
      if ansatz_lcu[i] in param_gates:
        padded.append(params[j])
        j+=1
      else:
        padded.append(None)
    return np.array(padded)

def projector1(wires):
  projector = np.zeros((2, 2))
  projector[1,1] = 1
  qml.QubitUnitary(projector, wires=wires)

def str2gate(gate):
  if gate == 'H':
    return qml.Hadamard
  elif gate == 'X':
    return qml.PauliX
  elif gate == 'Y':
    return qml.PauliY
  elif gate == 'Z':
    return qml.PauliZ
  elif gate == 'RX':
    return qml.RX
  elif gate == 'RY':
    return qml.RY
  elif gate == 'RZ':
    return qml.RZ
  elif gate == 'Projector1':
    return projector1
  else:
    assert 0, "Gate not support."

def controlStr(gate):
  if gate == 'RX':
    return qml.CNOT
  elif gate == 'RY':
    return qml.CY
  elif gate == 'RZ':
    return qml.CZ
  else:
    assert 0, "Gate not support."
    
def print_ansatz(ansatz, coeffs, wires):
  coeffs = padding(coeffs)
  @qml.qnode(dev1)
  def ansatz_construct(ansatz, coeffs, wires):
    ansatz = [a.numpy() for a in ansatz]
    coeffs = [c.numpy() for c in coeffs]
    wires = [w.numpy() for w in wires]

    for i, coeff in enumerate(coeffs):
      if not coeff:
        if ansatz[i] ==  'Problem':
          problem(wires[i])
        else:
          str2gate(ansatz[i])(wires=wires[i])
      else:
        str2gate(ansatz[i])(coeff, wires=wires[i])

    return qml.state()
  print(qml.draw(ansatz_construct)(ansatz, coeffs, wires))

## Parameter shift

In [27]:
num_wires = 1
dev1 = qml.device("default.qubit", wires=num_wires)

num_wires = 2
dev2 = qml.device("default.qubit", wires=num_wires)

In [5]:
def parameter_shift_term(qnode, params, i, j):
    
    shifted = params.copy()
    
    shifted[i, j] += np.pi/2
    forward = qnode(shifted)  # forward evaluation

    shifted[i, j] -= np.pi
    backward = qnode(shifted) # backward evaluation

    return 0.5 * (forward - backward)

def parameter_shift(qnode, params):
    gradients = np.zeros_like((params))
    for i in range(len(gradients)):
        for j in range(len(gradients[0])):
            gradients[i, j] += parameter_shift_term(qnode, params, i, j)

    return gradients
  
gate_list = ['R', 'S1', 'D1'] 

## Finite difference

In [6]:
def finite_term(qnode, params, i, j):
    
    eps = 0.01

    shifted = params.copy()
    
    shifted[i, j] += eps
    forward = qnode(shifted)  # forward evaluation

    shifted[i, j] -= 2*eps
    backward = qnode(shifted) # backward evaluation

    return (1/(2*eps)) * (forward - backward)

def finite_difference(qnode, params):
    gradients = np.zeros_like((params))
    for i in range(len(gradients)):
        for j in range(len(gradients[0])):
            gradients[i, j] += finite_term(qnode, params, i, j)

    return gradients

## Linear Combination of Unitaries(LCU)

In [7]:
def lcu_term(ansatz, params, wires, diff_index):
  qml.Hadamard(wires=1)
  qml.adjoint(qml.S)(wires=1)

  for i, gate in enumerate(ansatz):
    if gate in param_gates:
      if i == diff_index:
        controlStr(gate)(wires=[1, 0])
      str2gate(gate)(params[i], wires=wires[i])
    else:
      if gate == 'Problem':
        problem()
      else:
        str2gate(gate)(wires=wires[i])
    
  qml.Hadamard(wires=1)


def lcu_term_adjointed(ansatz, params, wires, diff_index):
  ansatz = [a for a in reversed(ansatz)]
  params = params[::-1]

  qml.Hadamard(wires=1)
  for i, gate in enumerate(ansatz):
    if gate in param_gates:
      str2gate(gate)(-params[i], wires=wires[i])
      if i == len(ansatz)-diff_index-1:
        controlStr(gate)(wires=[1, 0])
    else:
      if gate == 'Problem':
         qml.adjoint(problem)()
      else:
        str2gate(gate)(wires=wires[i])

  qml.S(wires=1)
  qml.Hadamard(wires=1)


@qml.qnode(dev2)
def lcu(ansatz, H, ham_wires, params, wires, diff_index):
  ansatz = [a.numpy() for a in ansatz]
  wires = [w.numpy() for w in wires]
  ham_wires = ham_wires.numpy()

  lcu_term(ansatz, params, wires, diff_index)
  str2gate(H)(wires=ham_wires)
  str2gate('Z')(wires=1)
  lcu_term_adjointed(ansatz, params, wires, diff_index)

  return qml.state()

def lcu_gradients(ansatz, H, ham_wires, ham_coeffs, params, wires):

  params = padding(params)

  gradients = []
  for i in range(len(ansatz)):
    if ansatz[i] in param_gates:
      gradients.append(sum([lcu(ansatz, H[h], ham_wires[h], params, wires, i)[0].numpy()*ham_coeffs[h] for h in range(len(H))]).real)

  return gradients

# **Problem** : Quantum gate estimation


Using $U3(1.1, 0.5, 2.3)$ as target gate

In [8]:
# problem gate 
def problem():
    qml.U3(1.44, 0.8, 2.1, wires=0)

Ansatz for estimating target gate

In [9]:
# guesting ansatz state
def ansatz(var):
    for wire in range(num_wires):
      qml.Hadamard(wires=wire)
      qml.RX(var[0+wire], wires=wire)
      qml.RY(var[1+wire], wires=wire)
      qml.RZ(var[2+wire], wires=wire)

Objective function <br> 
1. initial state is  $|0⟩$
2. apply target gate $U(\theta)|0⟩$ <br> 
the state will be $|\psi⟩ = a|0⟩+b|1⟩$ 
3. apply ansatz $A(\alpha)$  <br> 
the state will be $A(\alpha)|\psi⟩ = |\psi'⟩$
4. if $A(\alpha) = U(\theta)$ then  $|\psi'⟩= 1|0⟩ + 0|1⟩ = |0⟩$
5. so, we will minimize $b|1⟩$ to target $0|1⟩$ to make $A(\alpha) = U(\theta)$


In [28]:
# objective function
@qml.qnode(dev1)
def cost_function(var):
    for v in var: 
      ansatz(v)

    problem() # problem gate 

    return qml.expval(qml.Projector([1],wires=0)) # get amplitude of of |1>

Probability distribution of target gate 

In [29]:
# target result of problem gate
@qml.qnode(dev1)
def target():
    problem()
    return qml.probs(wires=[0])  # get target probability

Probability distribution of estimated gate 

In [30]:
# prediction circuit
@qml.qnode(dev1)
def prediction(var):
    for v in var:
      ansatz(v)
    return qml.probs(wires=[0])  # get prediction probability

In [31]:
print("Target state: ", target())

Target state:  [0.56521185 0.43478815]


In [37]:
np.random.seed(1)
num_wires = 1
num_layers = 2
var_init = 0.05*np.random.randn(num_layers, 3*num_wires)

In [38]:
print("Initial cost: ", cost_function(var_init))

Initial cost:  0.38420647954293236


Training with parameter shift rule and gradient descent optimizer

In [39]:
opt = GradientDescentOptimizer(0.01)

var = var_init.copy()
loss_plot = []

for it in range(1201):# while True:
    var, _cost = opt.step_and_cost(lambda v: cost_function(v), var, 
                                   grad_fn=lambda var: parameter_shift(cost_function, var)) 
    loss_plot.append(_cost)

    if it%100==0:
      print("Iter: {:5d} | Cost: {:0.11f} ".format(it, _cost))

Iter:     0 | Cost: 0.38420647954 
Iter:   100 | Cost: 0.07905926083 
Iter:   200 | Cost: 0.01260131677 
Iter:   300 | Cost: 0.00204853691 
Iter:   400 | Cost: 0.00034842518 
Iter:   500 | Cost: 0.00006112723 
Iter:   600 | Cost: 0.00001091990 
Iter:   700 | Cost: 0.00000197059 
Iter:   800 | Cost: 0.00000035761 
Iter:   900 | Cost: 0.00000006510 
Iter:  1000 | Cost: 0.00000001187 
Iter:  1100 | Cost: 0.00000000217 
Iter:  1200 | Cost: 0.00000000040 


In [40]:
opt = GradientDescentOptimizer(0.01)

var_fd = var_init.copy()
loss_plot_fd = []

for it in range(1201):# while True:
    var_fd, _cost = opt.step_and_cost(lambda v: cost_function(v), var_fd, 
                                   grad_fn=lambda var_fd: finite_difference(cost_function, var_fd)) 
    loss_plot_fd.append(_cost)

    if it%100==0:
      print("Iter: {:5d} | Cost: {:0.11f} ".format(it, _cost))

Iter:     0 | Cost: 0.38420647954 
Iter:   100 | Cost: 0.07906163434 
Iter:   200 | Cost: 0.01260209243 
Iter:   300 | Cost: 0.00204872124 
Iter:   400 | Cost: 0.00034846608 
Iter:   500 | Cost: 0.00006113608 
Iter:   600 | Cost: 0.00001092178 
Iter:   700 | Cost: 0.00000197099 
Iter:   800 | Cost: 0.00000035769 
Iter:   900 | Cost: 0.00000006511 
Iter:  1000 | Cost: 0.00000001187 
Iter:  1100 | Cost: 0.00000000217 
Iter:  1200 | Cost: 0.00000000040 


In [57]:
def ansatz_circuit_lcu(ansatz, var, wires):
  var = padding(var)
  for i, gate in enumerate(ansatz):
    if var[i]:
      str2gate(gate)(var[i], wires=wires[i])  
    else:
      str2gate(gate)(wires=wires[i])  


@qml.qnode(dev1)
def cost_function_lcu(ansatz, params, wires):
  ansatz = [a.numpy() for a in ansatz]
  wires = [w.numpy() for w in wires]

  ansatz_circuit_lcu(ansatz[:-1], params, wires[:-1])

  problem()

  return qml.expval(qml.Projector([1], wires=0))

@qml.qnode(dev1)
def prediction_lcu(ansatz, params, wires):
  ansatz = [a.numpy() for a in ansatz]
  wires = [w.numpy() for w in wires]

  ansatz_circuit_lcu(ansatz[:-1], params, wires[:-1])
  
  return qml.probs(wires=[0])

@qml.qnode(dev1)
def target_lcu(wires):
  problem()
  return qml.probs(wires=[0])

In [58]:
np.random.seed(1)

num_layers = 2
num_wires = 1

ansatz_lcu = ['H', 'RX', 'RY', 'RZ']*num_layers + ['Problem']
wires_lcu = [0, 0, 0, 0]*num_layers + [0]

H = ['Projector1']
ham_wires = [0]
ham_coeffs = [1.0]

params_init= 0.05*np.random.randn(1, 3*num_wires*num_layers)[0]

opt = GradientDescentOptimizer(0.01)

var_lcu = params_init.copy()
loss_plot_lcu  = []

for it in range(1201):# while True:
    var_lcu, _cost = opt.step_and_cost(lambda v: cost_function_lcu(ansatz_lcu, v, wires_lcu), var_lcu, 
                                   grad_fn=lambda var_lcu: lcu_gradients(ansatz_lcu, H, ham_wires, ham_coeffs, var_lcu, wires_lcu)) 
    loss_plot_lcu.append(_cost)

    if it%100==0:
      print("Iter: {:5d} | Cost: {:0.11f} ".format(it, _cost))

Iter:     0 | Cost: 0.38420647954 
Iter:   100 | Cost: 0.07905926083 
Iter:   200 | Cost: 0.01260131677 
Iter:   300 | Cost: 0.00204853691 
Iter:   400 | Cost: 0.00034842518 
Iter:   500 | Cost: 0.00006112723 
Iter:   600 | Cost: 0.00001091990 
Iter:   700 | Cost: 0.00000197059 
Iter:   800 | Cost: 0.00000035761 
Iter:   900 | Cost: 0.00000006510 
Iter:  1000 | Cost: 0.00000001187 
Iter:  1100 | Cost: 0.00000000217 
Iter:  1200 | Cost: 0.00000000040 


# **Loss comparison**

In [60]:
#@title 
dv = pd.DataFrame({"Iteration":range(len(loss_plot)), "Loss":loss_plot})
dv_fd = pd.DataFrame({"Iteration":range(len(loss_plot_fd)), "Loss":loss_plot_fd})
dv_lcu = pd.DataFrame({"Iteration":range(len(loss_plot_lcu)), "Loss":loss_plot_lcu})

fig = go.Figure()
fig.add_trace(go.Scatter(x=dv["Iteration"], y=dv["Loss"], name="Discrete parameter shift"))
fig.add_trace(go.Scatter(x=dv_fd["Iteration"], y=dv_fd["Loss"], mode="lines", name="Discrete finite difference", line = dict(width=2, dash='dash')))
fig.add_trace(go.Scatter(x=dv_lcu["Iteration"], y=dv_lcu["Loss"], mode="lines", name="Discrete LCU", line = dict(width=2, dash='dash')))
fig.update_layout(title_text='Loss comparison')
fig.show()

In [64]:
pd.DataFrame({"Params-shift Loss":dv['Loss'], "Finite difference":dv_fd['Loss'], "LCU":dv_lcu['Loss']}, index=dv['Iteration'])

,Params-shift Loss,Finite difference,LCU
Iteration,,,
0,0.38420647954293236,0.38420647954293236,0.38420647954293236
1,0.37947911086194586,0.37947918955597504,0.37947911086194586
2,0.3747745770899185,0.37477473370626957,0.3747745770899185
3,0.3700936802444943,0.37009391397167724,0.3700936802444943
4,0.36543720443572447,0.3654375144236765,0.36543720443572447
...,...,...,...
1196,4.235315799491615e-10,4.2367573832819527e-10,4.2353157994154867e-10
1197,4.1639176387224356e-10,4.1653361033128383e-10,4.163917638675389e-10
1198,4.093723218482991e-10,4.0951189336920115e-10,4.09372321843634e-10
